In [1]:
import numpy as np
import pandas as pd


In [61]:
data_path = "../../dataset/1/modified.csv"
res_path = "result.csv"

In [3]:
df = pd.read_csv(data_path)

In [4]:
df.head()

,text,status,char_len,text_processed,word_len
0,"In other words #katandandre, your food was cra...",not_cyberbullying,61,word katandandre food crapilicious mkr,9
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,115,aussietv white mkr theblock imacelebrityau tod...,14
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,60,xochitlsuckkks classy whore red velvet cupcake,9
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,103,jason_gio meh thanks head concern another angr...,18
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,103,rudhoeenglish isi account pretend kurdish acco...,18


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            47692 non-null  object
 1   status          47692 non-null  object
 2   char_len        47692 non-null  int64 
 3   text_processed  47682 non-null  object
 4   word_len        47692 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.8+ MB


In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split

In [36]:
df['status_num'] = df['status'].map({'not_cyberbullying':0, 'gender':1, 'religion':2, 'other_cyberbullying':3,
       'age':4, 'ethnicity':5})

In [37]:
X = df['text_processed']
y = df['status_num']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [39]:
y_train_exp = np.expand_dims(np.array(y_train),axis=-1)
y_test_exp = np.expand_dims(np.array(y_test),axis=-1)

In [40]:
cv = CountVectorizer()

cv.fit(X_train.values.astype('U')) # str will also work
X_train_trans = cv.transform(X_train.values.astype('U'))
X_test_trans = cv.transform(X_test.values.astype('str'))

In [41]:
!where python

/Applications/anaconda3/envs/miscEnv/bin/python


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [64]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve

In [68]:
# lr = LogisticRegression()
svc = SVC(kernel='sigmoid',gamma=1.0)
dtc = DecisionTreeClassifier(max_depth=5)
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()
knc = KNeighborsClassifier()
rf = RandomForestClassifier()
abc = AdaBoostClassifier(n_estimators=50, random_state = 32)
bc = BaggingClassifier(n_estimators=50, random_state=32)
etc = ExtraTreesClassifier(n_estimators = 50, random_state = 32)
gbdt = GradientBoostingClassifier(n_estimators = 50,random_state = 32)
xgb = XGBClassifier(n_estimators=50, random_state = 32)

In [ ]:
mdls = {
'svc':svc, # too much time
'dtc':dtc, 
'mnb':mnb,
# 'gnb':gnb, # sparse data issue
'bnb':bnb,
'knc':knc,
'rf':rf,
'abc':abc,
'bc':bc, # too much time
'etc':etc, # too much time
'gbdt':gbdt,
'xgb':xgb,
}

In [78]:
# mdls = {
#     # "lr":lr,
#     "mnb":mnb,
#     "knc":knc,
#     "dtc":dtc,
#     "abc":abc,
#     "gbdt":gbdt,
#     "xgb":xgb,
#     "bc":bc,
#     "svc":svc,
#     "etc":etc,
# }

mdls = {
# 'svc':svc, # too much time
'dtc':dtc, 
# 'mnb':mnb,
# 'gnb':gnb, # won't work
'bnb':bnb,
# 'knc':knc,
'rf':rf,
# 'abc':abc,
# 'bc':bc, # too much time
# 'etc':etc, # too much time
# 'gbdt':gbdt,
# 'xgb':xgb,
}


In [76]:
bnb.fit(X_train_trans,y_train)
y_pred2 = bnb.predict(X_test_trans)

In [70]:
print(accuracy_score(y_test,y_pred2))


0.7818429604780376


In [54]:
def predict_mdl(mdl, X_train, y_train, X_test, y_test):
    """Trains model and returns (accuracy,precision)"""
    mdl.fit(X_train, y_train)
    y_preds = mdl.predict(X_test)

    # print('hi')
    print(accuracy_score(y_test,y_preds))
    # print(confusion_matrix(y_test,y_preds))

    return accuracy_score(y_preds, y_test)

In [55]:
names = []
accuracies = []
# precisions = []
for name, mdl in mdls.items():
    print()
    print("---===---"*5)
    print(name)
    acc = predict_mdl(mdl, X_train_trans, y_train, X_test_trans, y_test)
    print(acc)
    names.append(name)
    accuracies.append(acc)



---===------===------===------===------===---
mnb
0.7818429604780376
0.7818429604780376

---===------===------===------===------===---
knc
0.7098228325820316
0.7098228325820316

---===------===------===------===------===---
dtc
0.6388510326029982
0.6388510326029982

---===------===------===------===------===---
abc
0.7985113743578992
0.7985113743578992

---===------===------===------===------===---
gbdt
0.8268162281161547
0.8268162281161547

---===------===------===------===------===---
xgb
0.8461054617884475
0.8461054617884475

---===------===------===------===------===---
bc
0.8229374148233567
0.8229374148233567

---===------===------===------===------===---
svc
0.5572911206625433
0.5572911206625433

---===------===------===------===------===---
etc
0.8249292378656043
0.8249292378656043


In [57]:
res_df = pd.DataFrame({"models_cv-1":names,"accuracy_cv-1":accuracies})
res_df

,models_cv-1,accuracy_cv-1
0,mnb,0.781843
1,knc,0.709823
2,dtc,0.638851
3,abc,0.798511
4,gbdt,0.826816
5,xgb,0.846105
6,bc,0.822937
7,svc,0.557291
8,etc,0.824929


In [62]:
res_df.to_csv(res_path, index=False)

In [79]:
names1 = []
accuracies1 = []
for name, mdl in mdls.items():
    print()
    print("---===---"*5)
    print(name)
    acc = predict_mdl(mdl, X_train_trans, y_train, X_test_trans, y_test)
    print(acc)
    names1.append(name)
    accuracies1.append(acc)



---===------===------===------===------===---
dtc
0.6388510326029982
0.6388510326029982

---===------===------===------===------===---
bnb
0.8229374148233567
0.8229374148233567

---===------===------===------===------===---
rf
0.821784254114687
0.821784254114687


In [87]:
# res_df = pd.concat([res_df,res_df2],ignore_index=True)


In [1]:
# res_df.to_csv(res_path)
res_df

NameError: name 'res_df' is not defined